In [15]:
import json 
from pprint import pprint
from llm_handler import LLMHandler
from config import FILE_PATHS
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [16]:
domain = "banking"
category = "adaptive_tool_use"

persona_path = FILE_PATHS["personas"].format(domain=domain)
scenario_path = FILE_PATHS["scenarios"].format(domain=domain, category=category)
tool_path = FILE_PATHS["tools"].format(domain=domain)


In [22]:
llm = LLMHandler().get_llm(model_name="gpt-4.1-2025-04-14", temperature=0.0)

with open(tool_path, "r") as f:
    tools = json.load(f)

with open(scenario_path, "r") as f:
    scenarios = json.load(f)
    
llm = llm.bind_tools(tools)

In [25]:
# call the llm with the tools
system_prompt = """
You are a helpful assistant that can use the tools given to help the user.
"""

query = "What is the balance of my account with the id 1234567890?"
query = "i want to check all the credit transactions for my account with the id 1234567890"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query},
]
response = llm.invoke(messages)
response.tool_calls

[{'name': 'get_transaction_history',
  'args': {'account_id': '1234567890', 'transaction_type': 'credit'},
  'id': 'call_b3Ct0kFEfwRIt6FUjwhuDzga',
  'type': 'tool_call'}]